In [1]:
import pandas as pd
import numpy as np

## Download and filter

In [7]:
RELATIVE_PATH_IN_CURATED = "../../../data/3. curated"
RELATIVE_PATH_IN_MODELLING = "../../../data/4. modelling"

In [8]:
curated_raw = pd.read_csv(f"{RELATIVE_PATH_IN_CURATED}/external prices.csv", index_col=0)
print(curated_raw.shape)

(6228, 181)


In [9]:
def filter_frame(df, type=False, start_year=False, end_year=False):
    df_mask = pd.Series(data=True, index=df.index)
    if (type):
        df_mask = df_mask & (df["housing: type"] == type)
    
    if (start_year):
        df_mask = df_mask & (df["year groups"] >= start_year)
    if (end_year):
        df_mask = df_mask & (df["year groups"] <= end_year)
    
    return df[df_mask]

In [10]:
def filter_columns(df, find=[], avoid=[], require_all=False):
    curr_columns = df.columns
    
    # filter any columns that contain the find_sub
    if (find):
        if (not require_all):
            curr_columns = [column for column in curr_columns if any(sub in column for sub in find)]
        else:
            curr_columns = [column for column in curr_columns if all(sub in column for sub in find)]

    # avoid any columns that contain the avoid_sub
    curr_columns = [column for column in curr_columns if all(not sub in column for sub in avoid)]

    return curr_columns

FEATURES = filter_columns(curated_raw, find=["economic:"], avoid=["%", "gdp quarterly"]) + \
           filter_columns(curated_raw, find=["housing:"]) + \
           filter_columns(curated_raw, find=["overseas:"]) + \
           filter_columns(curated_raw, find=["population:"]) + \
           filter_columns(curated_raw, find=["relationships:"]) + \
           filter_columns(curated_raw, find=["studying:"], avoid=["PT", "FT"]) + \
           filter_columns(curated_raw, find=["distance:"]) + \
           ["suburbs", "year groups"]

curated = curated_raw[FEATURES]

print(curated.shape)
curated.head(3)

(6228, 121)


,economic: median income,economic: median age of earners,economic: gini coefficient,economic: variable interest rate,economic: gdp per capita quarterly,economic: trimmed mean quarterly,economic: number of earners,economic: net inflation,economic: variable interest rate growth,housing: type,...,studying: tertiary total (%),studying: tafe total (%),studying: primary government (%),studying: primary total (%),distance: crow distance to cbd,distance: distance to cbd,distance: crow distance to cbd inv,distance: distance to cbd inv,suburbs,year groups
0,63415.3,39.8,0.5565,6.3825,0.625,0.675,21731.8,1.044823,-0.070,flat,...,0.0637,0.0236,0.0336,0.0436,4.535857,6702.5,0.081735,0.211996,Albert Park-Middle Park-West St Kilda,2.0
1,63415.3,39.8,0.5565,6.6325,0.725,0.675,21731.8,1.073584,0.039,flat,...,0.0637,0.0236,0.0336,0.0436,4.535857,6702.5,0.081735,0.211996,Albert Park-Middle Park-West St Kilda,3.0
2,63415.3,39.8,0.5565,7.0700,0.475,0.675,21731.8,1.102315,0.066,flat,...,0.0637,0.0236,0.0336,0.0436,4.535857,6702.5,0.081735,0.211996,Albert Park-Middle Park-West St Kilda,4.0


## Checking Linearities

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
Y_COLUMN = "housing: median growth avg"

def plot_subset(df, find=False, name="pairplot.png", display=True, root=False):
    y_column = Y_COLUMN

    if (find):
        print(f"using \"{find}\" to filter")

        # get the filtered df
        df = df.copy()
        df = df[filter_columns(df, find=[find, y_column])]

    # print the shape and ask to continue
    response = input(f"frame has {df.shape[1] - 1} columns, wish to proceed? (y)")
    if (response.lower() != "y"):
        return

    print(filter_columns(df, avoid=[y_column]))

    # graph
    sns.set_style('white')
    pairplot = sns.pairplot(df, y_vars=y_column, x_vars=list(set(df.columns) - set([y_column])), diag_kind=None)# #plot_kws={'alpha': 0.6})

    # save and show
    if (display):
        plt.show()

### Distance

In [24]:
plot_subset(filter_frame(curated, type="house"), "distance:")
plot_subset(filter_frame(curated, type="flat"), "distance:")

using "distance:" to filter


using "distance:" to filter


### Housing

In [25]:
plot_subset(filter_frame(curated, type="house"), "housing:")
plot_subset(filter_frame(curated, type="flat"), "housing:")

using "housing:" to filter


using "housing:" to filter


### Population (not total)

In [26]:
POP_COLUMNS = filter_columns(curated, find=["population: ", "growth"], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house")[POP_COLUMNS], name="population house")
plot_subset(filter_frame(curated, type="flat")[POP_COLUMNS], name="population flat")

### Population Total

In [27]:
POP_COLUMNS = filter_columns(curated, find=["population: ", "total"], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house")[POP_COLUMNS], name="population total house")
plot_subset(filter_frame(curated, type="flat")[POP_COLUMNS], name="population total flat")

### Studying

In [28]:
STUDY_COLUMNS = filter_columns(curated, find=["study", "%"], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house", start_year=22, end_year=23)[STUDY_COLUMNS], name="study house")
plot_subset(filter_frame(curated, type="flat", start_year=22, end_year=23)[STUDY_COLUMNS], name="study flat")

### Relationships

In [29]:
RELATIONSHIPS_COLUMNS = filter_columns(curated, find=["relationships", "%"], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house", start_year=22, end_year=23)[RELATIONSHIPS_COLUMNS], name="relationships house")
plot_subset(filter_frame(curated, type="flat", start_year=22, end_year=23)[RELATIONSHIPS_COLUMNS], name="relationships flat")

### Economic

In [30]:
ECONOMIC_COLUMNS = filter_columns(curated, find=["economic: "], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house", start_year=22, end_year=23)[ECONOMIC_COLUMNS], name="relationships house")
plot_subset(filter_frame(curated, type="flat", start_year=22, end_year=23)[ECONOMIC_COLUMNS], name="relationships flat")

### Overseas

In [31]:
OVERSEAS_COLUMNS = filter_columns(curated, find=["oversea"], require_all=True) + filter_columns(curated, find=["housing: median growth"])

plot_subset(filter_frame(curated, type="house", start_year=22, end_year=23)[OVERSEAS_COLUMNS], name="overseas house")
plot_subset(filter_frame(curated, type="flat", start_year=22, end_year=23)[OVERSEAS_COLUMNS], name="overseas flat")

## Outliers

In [32]:
# filter out spots less than 200 (to noisy)
curated = curated[curated["housing: count"] >= 200]

## Transformations

In [46]:
# do the log transforms
LOG_FEATURES = ["housing: previous count", "housing: count"] + \
               ["studying: tertiary total (%)", "studying: primary other (%)", "studying: tafe total (%)"] + \
               ["economic: number of earners"] + \
               ["population: total"] + \
               ["overseas: 5 years (%)"] + \
               ["distance: distance to cbd", "distance: crow distance to cbd"]

curated_final = curated.copy()
for feature in LOG_FEATURES:
    curated_final.loc[:, feature + " log"] = curated[feature].apply(np.log)
    curated_final.rename(columns={feature: feature + " norm"}, inplace=True)

In [47]:
plot_subset(filter_frame(curated_final, type="house")[filter_columns(curated_final, find=["log", "housing: median"])], name="transformation housing", root=False)
plot_subset(filter_frame(curated_final, type="flat")[filter_columns(curated_final, find=["log", "housing: median"])], name="transformation flat", root=False)

## Interaction

In [48]:
def get_interaction(df, interaction_pairs):
    # get all the interaction pairs
    for column_1, column_2 in interaction_pairs:
        new_name = f"interaction: ({column_1}) & ({column_2})"
        df[new_name] = df[column_1] * df[column_2]
    
    return df

In [49]:
POP_COLUMNS = filter_columns(curated_final, ["population: ", "growth", "0", "9"], ["interaction"], require_all=True) + filter_columns(curated_final, ["population:", "total"], ["norm", "interaction"], require_all=True)

# will get interaction between
# - all population growths and distance / median income
# - quarterly growths with distance / median income
# - all population growths and teritary
interaction_pairs = [(x, y) for x in filter_columns(curated_final, ["distance:", "economic: median income"], ["inv", "norm", "interaction"]) for y in POP_COLUMNS] + \
                    [(x, y) for x in filter_columns(curated_final, ["distance:", "economic: median income"], ["inv", "norm", "interaction"]) for y in ['economic: trimmed mean quarterly', 'economic: gdp per capita quarterly']] + \
                    [(x, y) for x in filter_columns(curated_final, ["studying:", "tertiary", "log"], ["interaction"], require_all=True) for y in POP_COLUMNS]

curated_final = get_interaction(curated_final, interaction_pairs)

## Fitting the model

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [51]:
##### FOR all variables that can actually work ###########

# include the following
# - information about the region that doesn't change (economic)
# - types of beds and the previous, price growth, count growth, log count, price
# - number of people overseas 5 years ago
# - population growth by each demographic
# - population total count log
# - percentage of the population studying and what not
FEATURES = filter_columns(curated_final, find=["housing"], avoid=["norm", "median growth", "housing: count", "housing: median", "interaction", "avg"]) + \
           POP_COLUMNS + \
           filter_columns(curated_final, find=["economic", "quarterly"], avoid=["distance"], require_all=True) + \
           filter_columns(curated_final, find=["interaction", "population"], avoid=["distance"], require_all=True)
           

CAT_FEATURES = ["housing: type"]

len(FEATURES)

42

### Functions

In [52]:
def calculate_aic(model, X, y):
    """
    Calculate the Akaike Information Criterion (AIC) for a given scikit-learn model.

    Parameters:
    - model: A fitted scikit-learn model that implements the `predict` method.
    - X: Features (numpy array or pandas DataFrame) used for fitting the model.
    - y: True target values (numpy array or pandas Series).

    Returns:
    - aic: The Akaike Information Criterion value.
    """
    # Ensure the model has been fitted
    if not hasattr(model, 'predict'):
        raise ValueError("The model must implement the `predict` method and be fitted.")

    # Number of observations
    n = len(y)

    # Number of parameters (coefficients + intercept)
    if hasattr(model, 'coef_'):
        # Linear models (e.g., LinearRegression)
        k = model.coef_.shape[0] + 1  # Coefficients + Intercept
    else:
        # Other models
        k = len(model.get_params())

    # Predicted values
    y_pred = model.predict(X)

    # Calculate residual sum of squares
    residual_sum_of_squares = np.sum((y - y_pred) ** 2)

    # Calculate AIC
    aic = n * np.log(residual_sum_of_squares / n) + 2 * k
    return aic

In [53]:
def forward_selection_with_validation(X_select_train, y_select_train, X_select_val, y_select_val, validation_size=0.2, random_state=42, use_aic=True):
    """
    Perform forward selection using a validation set to choose the best set of features for linear regression.
    
    Args:
    X: Features (DataFrame)
    y: Target variable (Series)
    validation_size: Fraction of data to use as validation set (default 0.2)
    random_state: Random seed for reproducibility (default 42)
    
    Returns:
    best_features: List of selected features based on validation set performance
    final_model: Trained linear regression model using the selected features
    """

    # Rename
    X_train, y_train, X_val, y_val = X_select_train, y_select_train, X_select_val, y_select_val
    
    # Initialize variables
    initial_features = []
    remaining_features = list(X_train.columns)
    best_features = []
    best_score = float('inf')  # Start with a very high RMSE
    
    # Iterate through all features to add one by one
    while remaining_features:
        scores_with_candidates = []
        
        # Try adding each remaining feature to the model
        for candidate in remaining_features:
            # Features to include in the model
            features_to_test = initial_features + [candidate]
            
            # Fit a linear regression model using the current feature set
            model = LinearRegression()
            model.fit(X_train[features_to_test], y_train)
            
            if (not use_aic):
                # Predict and calculate RMSE on the validation set
                y_val_pred = model.predict(X_val[features_to_test])
                rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

                # Store the result: (rmse, feature_name)
                scores_with_candidates.append((rmse, candidate))
                
            else:
                # get the AIC
                aic = calculate_aic(model, X_val[features_to_test], y_val)
                scores_with_candidates.append((aic, candidate))
        
        # Sort by RMSE, select the best candidate feature
        scores_with_candidates.sort(reverse=False)
        best_new_score, best_new_feature = scores_with_candidates[0]
        
        # If the score improves, update the best features list
        if best_new_score < best_score:
            initial_features.append(best_new_feature)
            remaining_features.remove(best_new_feature)
            best_features = initial_features.copy()
            best_score = best_new_score
            #if (use_aic):
            #    print(f"Selected '{best_new_feature}' with AIC: {best_new_score:.4f}")
            #else:
            #    print(f"Selected '{best_new_feature}' with RMSE: {best_new_score:.4f}")
        else:
            break  # Stop if adding a new feature doesn't improve the score
    
    return best_features, best_score


In [54]:
def get_mask(df, start_year, end_year=False):
    if (not end_year):
        mask = df["year groups"] == start_year
    else:
        mask = ((df["year groups"] >= start_year) &
                (df["year groups"] <= end_year))
    return mask

In [55]:
TEST_COLUMN = "housing: median growth"

def get_data(df, start_year, end_year, test_year, features):

    # remove `all`` properties
    df_filt = df.copy()[(df["housing: type"] == "house")] #& (curated_final["housing: beds"] == "3")]
    not_na_mask = ~df_filt[features + [TEST_COLUMN]].isna().any(axis=1)

    # test mask
    test_mask = get_mask(df_filt, test_year) & not_na_mask

    # get the mask forward selection
    selection_train_masks = [get_mask(df_filt, start_year, end_year=end_year-1) & not_na_mask for start_year in range(start_year, end_year)]
    selection_val_mask = get_mask(df_filt, end_year) & not_na_mask

    # get the train test features
    X = df_filt[features]
    y = df_filt[TEST_COLUMN]

    # Step 3: One-Hot Encode the categorical features
    X = pd.get_dummies(X, columns=list(set(CAT_FEATURES) & set(features)), drop_first=True)

    # split the train and test
    X_train, y_train, X_test, y_test = X[selection_train_masks[0]], y[selection_train_masks[0]], X[test_mask], y[test_mask]

    # store in dictionary
    data_dict = {
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test, 
        "y_test": y_test,
        "X": X,
        "y": y,
        "train_masks": selection_train_masks,
        "val_mask": selection_val_mask
        }
    return data_dict

In [56]:
def get_train_test(df, start_year, end_year, test_year, features):
    data_dict = get_data(df, start_year, end_year, test_year, features)

    # get the datasets
    train_mask = data_dict["train_masks"][0]
    val_mask = data_dict["val_mask"]
    X = data_dict["X"]
    y = data_dict["y"]
    X_test = data_dict["X_test"]
    y_test = data_dict["y_test"]

    # split the validation
    X_select_train, y_select_train, X_select_val, y_select_val = X[train_mask], y[train_mask], X[val_mask], y[val_mask]

    # get the best features
    best_features, score = forward_selection_with_validation(X_select_train, y_select_train, X_select_val, y_select_val, use_aic=True)

    # get the best feature mask and split
    X_train = X[train_mask][best_features]
    y_train = y[train_mask]
    X_test = X_test[best_features]

    return X_train, y_train, X_test, y_test, best_features

In [60]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_rrf(X_train, y_train, X_test, y_test):   
    # Train a Random Forest Regressor
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=8)
    rf_regressor.fit(X_train, y_train)

    # score the model
    y_test_pred = rf_regressor.predict(X_test)

    # scoring
    r2_test = r2_score(y_test, y_test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    print(f"Random Forest Model Performance:")
    print(f"Testing  R^2: {r2_test:.4f}, RMSE: {rmse_test:.4f}")

    # Baseline metrics (Median predictor)
    median_test_pred = np.full_like(y_test, np.median(y_train))
    median_r2_test = r2_score(y_test, median_test_pred)
    median_rmse_test = np.sqrt(mean_squared_error(y_test, median_test_pred))
    print(f"\nBaseline Model (Median Predictor) Performance:")
    print(f"Testing  R^2: {median_r2_test:.4f}, RMSE: {median_rmse_test:.4f}")

    return r2_test, median_r2_test

In [59]:
def evaluate(X_train, y_train, X_test, y_test, RRF=False):
    if (RRF):
        return evaluate_rrf(X_train, y_train, X_test, y_test)

    # train and test
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    # Predict target values using the test set
    y_pred = lin_reg.predict(X_test)

    # Calculate evaluation metrics
    mse_linear = mean_squared_error(y_test, y_pred)
    r2_linear = r2_score(y_test, y_pred)

    # baseline data2
    baseline_pred = [y_train.mean()] * len(y_test)

    # baseline score
    mse_baseline = mean_squared_error(y_test, baseline_pred)
    r2_baseline = r2_score(y_test, baseline_pred)

    # zero R
    zero = [0] * len(y_test)

    # baseline score
    mse_zero = mean_squared_error(y_test, zero)
    r2_zero = r2_score(y_test, zero)

    # Step 9: Print comparison results
    print(f"Linear Model - RMSE: {np.sqrt(mse_linear):.3f}, R-squared: {r2_linear:.3f}")
    print(f"Baseline Model - RMSE: {np.sqrt(mse_baseline):.3f}, R-squared: {r2_baseline:.3f}")
    print(f"Zero Model - RMSE: {np.sqrt(mse_zero):.3f}, R-squared: {r2_zero:.3f}")

    return r2_linear, r2_baseline, r2_zero

### Getting the best features

In [66]:
from collections import Counter

TRAINING_SIZE = 3

max_year = int(curated_final["year groups"].max())
min_year = int(curated_final["year groups"].min())

best_features_list = []
print("up to year: ", end="")
for curr_start_year in range(min_year, max_year-TRAINING_SIZE+1):
    print(f"{curr_start_year}, ", end="")

    # get the data dict
    _a, _b, _c, _d, best_features = get_train_test(curated_final, curr_start_year, curr_start_year+TRAINING_SIZE, curr_start_year+TRAINING_SIZE+1, FEATURES)

    best_features_list.append(best_features)

flattened_features = [item for sublist in best_features_list for item in sublist]
features_dict = dict(Counter(flattened_features))
#sorted(features_dict.items(), key=lambda x: x[1])

# get the best features, that occur in at least `count thresh` train test cycles
COUNT_THRESH = 4
SELECT_FEATURES_NO = [column_key for column_key, count in features_dict.items() if count >= COUNT_THRESH]

up to year: 2, 

3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 

In [65]:
for i in range(2, 17):
    print(i)
    ADD = i

    START_YEAR = 0 + ADD
    END_YEAR = 4 + ADD
    TEST_YEAR = 5 + ADD

    dd = get_data(curated_final, START_YEAR, END_YEAR, TEST_YEAR, SELECT_FEATURES_NO)
    X_train, y_train, X_test, y_test = dd["X_train"], dd["y_train"], dd["X_test"], dd["y_test"]

    evaluate(X_train, y_train, X_test, y_test, RRF=True)

2


NameError: name 'features_dict' is not defined

In [ ]:
for i in range(2, 17):
    print(i)
    ADD = i

    START_YEAR = 0 + ADD
    END_YEAR = 4 + ADD
    TEST_YEAR = 5 + ADD

    dd = get_data(curated_final, START_YEAR, END_YEAR, TEST_YEAR, SELECT_FEATURES_NO)
    X_train, y_train, X_test, y_test = dd["X_train"], dd["y_train"], dd["X_test"], dd["y_test"]

    evaluate(X_train, y_train, X_test, y_test, RRF=False)

### Predictions

In [61]:
SELECT_FEATURES = ["housing: previous price", "housing: previous count growth", "housing: previous growth", "housing: previous count log"] + \
                  ["population: total log", "economic: trimmed mean quarterly"] + \
                  ["interaction: (studying: tertiary total (%) log) & (population: total log)", "interaction: (economic: median income) & (economic: trimmed mean quarterly)"]
SELECT_FEATURES

['housing: previous price',
 'housing: previous count growth',
 'housing: previous growth',
 'housing: previous count log',
 'population: total log',
 'economic: trimmed mean quarterly',
 'interaction: (studying: tertiary total (%) log) & (population: total log)',
 'interaction: (economic: median income) & (economic: trimmed mean quarterly)']

In [62]:
train_data = filter_frame(curated_final, start_year=21, end_year=23)
test_data = pd.read_csv(f"{RELATIVE_PATH_IN_MODELLING}/forecast_test.csv", index_col=0)

X_train, y_train, X_test = train_data[SELECT_FEATURES], train_data["housing: median growth"], test_data[SELECT_FEATURES]

In [63]:
# train and test
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Predict target values using the test set
y_pred = lin_reg.predict(X_test)

# final frame
results = pd.DataFrame(data={"predicitions": y_pred, "suburbs": test_data["suburbs"].values, "housing: type": test_data["housing: type"].values})
results

,predicitions,suburbs,housing: type
0,0.114625,Albert Park-Middle Park-West St Kilda,flat
1,0.056722,Albert Park-Middle Park-West St Kilda,house
2,0.081970,Altona,flat
3,0.103797,Altona,house
4,0.114085,Armadale,flat
...,...,...,...
274,0.044602,Williamstown,house
275,0.079324,Wodonga,flat
276,0.063490,Wodonga,house
277,0.047381,Yarraville-Seddon,flat


In [64]:
best = results.nlargest(10, "predicitions")
best

,predicitions,suburbs,housing: type
46,0.286198,CBD-St Kilda Rd,flat
228,0.234282,Southbank,flat
53,0.228233,Carlton-Parkville,flat
185,0.215569,North Melbourne-West Melbourne,flat
58,0.199992,Caulfield,flat
82,0.197258,Docklands,flat
224,0.171468,South Melbourne,flat
209,0.153960,Richmond-Burnley,flat
116,0.152702,Footscray,flat
30,0.148258,Box Hill,flat
